In [ ]:
# Import Dependencies
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import cm
import math, re, os
import pandas as pd
import numpy as np
import random
import tensorflow as tf
import tensorflow_addons as tfa

print("Tensorflow version " + tf.__version__)

In [ ]:
# Find number of TPU cores for setup --> determines batch size
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    tpu_strat = tf.distribute.experimental.TPUStrategy(tpu)
else:
    tpu_strat = tf.distribute.get_strategy() 
    
REPLICAS = tpu_strat.num_replicas_in_sync

print("REPLICAS: ", REPLICAS) #Number of TPU cores available for use

In [ ]:
# Getting Path for Kaggle Dataset
from kaggle_datasets import KaggleDatasets

initial_path = KaggleDatasets().get_gcs_path('tpu-getting-started')
print(initial_path)

In [ ]:
# Reading Training, Validation and Testing Images

IMAGE_SIZE = [512, 512] #size of image
WIDTH = IMAGE_SIZE[0]
HEIGHT = IMAGE_SIZE[1]
CHANNELS = 3 # RGB colors
BATCH_SIZE = 16 * REPLICAS #Some Number Multiplied with # of TPU Cores so all of them can be at use
                           #Batch Size = 128 --> each TPU core will work with 16 images at a time

file_path = initial_path + '/tfrecords-jpeg-512x512'
AUTO = tf.data.experimental.AUTOTUNE

train_file = tf.io.gfile.glob(file_path + '/train/*.tfrec')
valid_file = tf.io.gfile.glob(file_path + '/val/*.tfrec')
test_file = tf.io.gfile.glob(file_path + '/test/*.tfrec') 

# target/types of flowers in dataset --> 104 total
CLASSES = ['pink primrose',    'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',     'wild geranium',     'tiger lily',           'moon orchid',              'bird of paradise', 'monkshood',        'globe thistle',         # 00 - 09
           'snapdragon',       "colt's foot",               'king protea',      'spear thistle', 'yellow iris',       'globe-flower',         'purple coneflower',        'peruvian lily',    'balloon flower',   'giant white arum lily', # 10 - 19
           'fire lily',        'pincushion flower',         'fritillary',       'red ginger',    'grape hyacinth',    'corn poppy',           'prince of wales feathers', 'stemless gentian', 'artichoke',        'sweet william',         # 20 - 29
           'carnation',        'garden phlox',              'love in the mist', 'cosmos',        'alpine sea holly',  'ruby-lipped cattleya', 'cape flower',              'great masterwort', 'siam tulip',       'lenten rose',           # 30 - 39
           'barberton daisy',  'daffodil',                  'sword lily',       'poinsettia',    'bolero deep blue',  'wallflower',           'marigold',                 'buttercup',        'daisy',            'common dandelion',      # 40 - 49
           'petunia',          'wild pansy',                'primula',          'sunflower',     'lilac hibiscus',    'bishop of llandaff',   'gaura',                    'geranium',         'orange dahlia',    'pink-yellow dahlia',    # 50 - 59
           'cautleya spicata', 'japanese anemone',          'black-eyed susan', 'silverbush',    'californian poppy', 'osteospermum',         'spring crocus',            'iris',             'windflower',       'tree poppy',            # 60 - 69
           'gazania',          'azalea',                    'water lily',       'rose',          'thorn apple',       'morning glory',        'passion flower',           'lotus',            'toad lily',        'anthurium',             # 70 - 79
           'frangipani',       'clematis',                  'hibiscus',         'columbine',     'desert-rose',       'tree mallow',          'magnolia',                 'cyclamen ',        'watercress',       'canna lily',            # 80 - 89
           'hippeastrum ',     'bee balm',                  'pink quill',       'foxglove',      'bougainvillea',     'camellia',             'mallow',                   'mexican petunia',  'bromelia',         'blanket flower',        # 90 - 99
           'trumpet creeper',  'blackberry lily',           'common tulip',     'wild rose']                                                                                                                                               # 100 - 103

# Helper Functions to Read Data
def decode_image(data_image):
    image = tf.image.decode_jpeg(data_image, channels=CHANNELS)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "class": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['class'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "id": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['id']
    return image, idnum # returns a dataset of image(s)

def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False 

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

In [ ]:
#Functions for Data Augmentation Techniques

#random seed
seed = 123

def random_flip_lr(image, label):
    # Random flip left right
    image = tf.image.random_flip_left_right(image)
    return image, label

def random_brightness(image, label):
    # Change Brightness
    image = tf.image.random_brightness(image, 0.6, seed = seed)
    return image, label

def random_saturation(image, label):
    # Change Saturation
    image = tf.image.random_saturation(image, 3, 5, seed = seed)
    return image, label
    
def random_contrast(image, label):
    # Change Constrast
    image = tf.image.random_contrast(image, 0.3, 0.5, seed = seed)
    return image, label

def random_hue(image, label):
    # Change Hue
    image = tf.image.random_hue(image, 0.5, seed = seed)
    return image, label
    
def make_blur(image, label):
    # Blur Image
    image = tfa.image.mean_filter2d(image, filter_shape = 10)
    return image, label

def random_flip_all(image, label):
    # Random flip in all directions (left right up down)
    image = tf.image.random_flip_left_right(image, seed = seed)
    image = tf.image.random_flip_up_down(image, seed = seed)
    return image, label

# Randomly Blockout Parts of Some Images (images to have blockouts are chosen at random)
def random_blockout(image, label, sw=0.1, sh=0.2, rl=0.4):
    decider=random.random()
    if decider>=0.3:
        
        total_area = tf.cast(HEGHT*WIDTH, tf.float32)

        erase_width = tf.cast(tf.round(tf.sqrt(total_area * sw * rl)), tf.int32)
        erase_height = tf.cast(tf.round(tf.sqrt(total_area * sh / rl)), tf.int32)

        erase_height_2 = tf.minimum(erase_height, HEIGHT)
        erase_width_2 = tf.minimum(erase_length,WIDTH)

        erase_height_3 = tf.random.uniform(shape=[], minval=erase_height, maxval=erase_height_2, dtype=tf.int32)
        erase_width_3 = tf.random.uniform(shape=[], minval=erase_width, maxval=erase_width_2, dtype=tf.int32)

        erase_area = tf.zeros(shape=[erase_height_3, erase_width_3, CHANNELS])
        erase_area = tf.cast(erase_area, tf.uint8)

        pad_height = HEIGHT - erase_height_3
        pad_top = tf.random.uniform(shape=[], minval=0, maxval=pad_height, dtype=tf.int32)
        pad_bottom = pad_height - pad_top

        pad_width = WIDTH - erase_width_3
        pad_left = tf.random.uniform(shape=[], minval=0, maxval=pad_width, dtype=tf.int32)
        pad_right = pad_width - pad_left

        erase_mask = tf.pad([erase_area], [[0,0],[pad_top, pad_bottom], [pad_left, pad_right], [0,0]], constant_values=1)
        erase_mask = tf.squeeze(erase_mask, axis=0)
        erased_image = tf.multiply(tf.cast(image,tf.float32), tf.cast(erase_mask, tf.float32))

        return tf.cast(erased_image, image.dtype), label
    else:
        return tf.cast(image, image.dtype), label
    

# Read in Train Dataset and Apply Augmentation Techniques
def generate_train_df():
    df = load_dataset(train_file, labeled=True)
    
    # DATA AUGMENTATION
    #simple augmentation (random flipping) --> worked best
    df = df.map(random_flip_lr, num_parallel_calls=AUTO)
    
    # tried other augmentation methods -- didn't work out as well
#     df = df.map(random_brightness, num_parallel_calls=AUTO)
#     df = df.map(random_saturation, num_parallel_calls=AUTO)
#     df = df.map(random_contrast, num_parallel_calls=AUTO)
#     df = df.map(random_hue, num_parallel_calls=AUTO)
#     df = df.map(make_blur, num_parallel_calls=AUTO)
#     df = df.map(random_blockout, num_parallel_calls=AUTO)
    
    df = df.repeat()
    df = df.shuffle(2048)
    df = df.batch(BATCH_SIZE)
    df = df.prefetch(AUTO)
    return df

# Read in Original Train Dataset for EDA & Visualization purposes
def get_original_train(ordered=True):
    df = load_dataset(train_file, labeled=True, ordered=ordered)
    df = df.batch(BATCH_SIZE)
    df = df.cache()
    df = df.prefetch(AUTO)
    return df

# Read in Validation Dataset
def generate_valid_df(ordered=False):
    df = load_dataset(valid_file, labeled=True, ordered=ordered)
    df = df.batch(BATCH_SIZE)
    df = df.cache()
    df = df.prefetch(AUTO)
    return df

# Read in Test Dataset
def generate_test_df(ordered=False):
    df = load_dataset(test_file, labeled=False, ordered=ordered)
    df = df.batch(BATCH_SIZE)
    df = df.prefetch(AUTO)
    return df

## Exploratory Data Analysis & Data Visualization

In [ ]:
# Print Size (Image Count) of Each Dataset

# Train data
original_train = get_original_train()
train_count = np.sum([int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in train_file])
train_labels = next(iter(original_train.unbatch().map(lambda image, label: label).batch(train_count))).numpy()

#valid data
valid_data = generate_valid_df()
valid_count = np.sum([int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in valid_file])
valid_labels = next(iter(valid_data.unbatch().map(lambda image, label: label).batch(valid_count))).numpy()

#test data
test_data = generate_test_df()
test_count = np.sum([int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in test_file])

print('Train Images Count:', train_count)
print('Validation Images Count:', valid_count)
print('Test Images Count:', test_count)

In [ ]:
# Visualization Utility Functions

def batch_to_numpy_images_and_labels(data):
    images, labels = data
    numpy_images = images.numpy()
    numpy_labels = labels.numpy()
    if numpy_labels.dtype == object: # binary string in this case,
                                     # these are image ID strings
        numpy_labels = [None for _ in enumerate(numpy_images)]
    # If no labels, only image IDs, return None for labels (this is
    # the case for test data)
    return numpy_images, numpy_labels

def title_from_label_and_target(label, correct_label):
    if correct_label is None:
        return CLASSES[label], True
    correct = (label == correct_label)
    return "{} [{}{}{}]".format(CLASSES[label], 'OK' if correct else 'NO', u"\u2192" if not correct else '',
                                CLASSES[correct_label] if not correct else ''), correct

def display_one_flower(image, title, subplot, red=False, titlesize=16):
    plt.subplot(*subplot)
    plt.axis('off')
    plt.imshow(image)
    if len(title) > 0:
        plt.title(title, fontsize=int(titlesize) if not red else int(titlesize/1.2), color='red' if red else 'black', fontdict={'verticalalignment':'center'}, pad=int(titlesize/1.5))
    return (subplot[0], subplot[1], subplot[2]+1)
    
def display_batch_of_images(databatch, predictions=None):
    """This will work with:
    display_batch_of_images(images)
    display_batch_of_images(images, predictions)
    display_batch_of_images((images, labels))
    display_batch_of_images((images, labels), predictions)
    """
    # data
    images, labels = batch_to_numpy_images_and_labels(databatch)
    if labels is None:
        labels = [None for _ in enumerate(images)]
        
    # auto-squaring: this will drop data that does not fit into square
    # or square-ish rectangle
    rows = int(math.sqrt(len(images)))
    cols = len(images)//rows
        
    # size and spacing
    FIGSIZE = 13.0
    SPACING = 0.1
    subplot=(rows,cols,1)
    if rows < cols:
        plt.figure(figsize=(FIGSIZE,FIGSIZE/cols*rows))
    else:
        plt.figure(figsize=(FIGSIZE/rows*cols,FIGSIZE))
    
    # display
    for i, (image, label) in enumerate(zip(images[:rows*cols], labels[:rows*cols])):
        title = '' if label is None else CLASSES[label]
        correct = True
        if predictions is not None:
            title, correct = title_from_label_and_target(predictions[i], label)
        dynamic_titlesize = FIGSIZE*SPACING/max(rows,cols)*40+3 # magic formula tested to work from 1x1 to 10x10 images
        subplot = display_one_flower(image, title, subplot, not correct, titlesize=dynamic_titlesize)
    
    #layout
    plt.tight_layout()
    if label is None and predictions is None:
        plt.subplots_adjust(wspace=0, hspace=0)
    else:
        plt.subplots_adjust(wspace=SPACING, hspace=SPACING)
    plt.show()

In [ ]:
# Display some training data images
display_batch_of_images(next(iter(original_train.unbatch().batch(9))))

In [ ]:
# Display some validation data images
display_batch_of_images(next(iter(valid_data.unbatch().batch(9))))

In [ ]:
# Display some test data images
display_batch_of_images(next(iter(test_data.unbatch().batch(9))))

In [ ]:
# Display some training data images after augmentation (flip left/right)
new_train = generate_train_df()
display_batch_of_images(next(iter(new_train.unbatch().batch(9))))

In [ ]:
# See Frequency of Classes in Dataset - Checking for Class Balance/Imbalance
train_label_counter = np.asarray([[label, (train_labels == index).sum()] for index, label in enumerate(CLASSES)])
valid_label_counter = np.asarray([[label, (valid_labels == index).sum()] for index, label in enumerate(CLASSES)])

train_label_counter = pd.DataFrame(train_label_counter)
valid_label_counter = pd.DataFrame(valid_label_counter)

train_label_counter[1] = train_label_counter[1].astype('float')
valid_label_counter[1] = valid_label_counter[1].astype('float')


fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(20, 60))

ax1 = sns.barplot(x=train_label_counter[1], y=train_label_counter[0], order=CLASSES, ax=ax1, palette='coolwarm')
ax1.set_title('Train', fontsize=25)
ax1.tick_params(labelsize=16)

ax2 = sns.barplot(x=valid_label_counter[1], y=valid_label_counter[0], order=CLASSES, ax=ax2, palette='YlOrBr')
ax2.set_title('Validation', fontsize=25)
ax2.tick_params(labelsize=16)

plt.show()

## Building the Model

In [ ]:
EPOCHS = 30 
# Learning Rate Schedule for Fine Tuning #
def exponential_lr(epoch,
                   start_lr = 0.00001, min_lr = 0.00001, max_lr = 0.00005,
                   rampup_epochs = 5, sustain_epochs = 0,
                   exp_decay = 0.8):

    def lr(epoch, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay):
        # linear increase from start to rampup_epochs
        if epoch < rampup_epochs:
            lr = ((max_lr - start_lr) /
                  rampup_epochs * epoch + start_lr)
        # constant max_lr during sustain_epochs
        elif epoch < rampup_epochs + sustain_epochs:
            lr = max_lr
        # exponential decay towards min_lr
        else:
            lr = ((max_lr - min_lr) *
                  exp_decay**(epoch - rampup_epochs - sustain_epochs) +
                  min_lr)
        return lr
    return lr(epoch,
              start_lr,
              min_lr,
              max_lr,
              rampup_epochs,
              sustain_epochs,
              exp_decay)

lr_callback = tf.keras.callbacks.LearningRateScheduler(exponential_lr, verbose=True)

rng = [i for i in range(EPOCHS)]
y = [exponential_lr(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))

In [ ]:
# Creating the Model
with tpu_strat.scope(): 
    # Models we tested
    #pretrained_model = tf.keras.applications.VGG16
    #pretrained_model = tf.keras.applications.MobileNetV2
    #pretrained_model = tf.keras.applications.InceptionResNetV2
        
    #best performing model
    pretrained_model = tf.keras.applications.ResNet101V2(
        include_top=False , # remove top layer
        weights='imagenet', # pre-trained weights on imagenet dataset
        input_shape=[*IMAGE_SIZE, CHANNELS]
    )
    
    pretrained_model.trainable = True # if True retrain the weights
    
    model = tf.keras.Sequential([
        pretrained_model, #Base pretrained on ImageNet to extract features from images
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(len(CLASSES), activation='softmax') #softmax for multiple classification
    ])

In [ ]:
model.compile(
    optimizer='nadam', #compared with adam optimizer --> nadam performed better
    loss = 'sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],
)

# Visualizing Model
model.summary()

tf.keras.utils.plot_model(model, show_shapes=True)

In [ ]:
# Print Data Shapes
print("Training:", new_train)
print ("Validation:", valid_data)
print("Test:", test_data)

## Fit Model

In [ ]:
# fitting the model
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
STEPS_PER_EPOCH = train_count // BATCH_SIZE

history = model.fit(
    new_train,
    validation_data=valid_data,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=[lr_callback, early_stopping],
)

In [ ]:
# Helper Function to Display Model Results
def display_training_curves(training, validation, title, subplot):
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    #ax.set_ylim(0.28,1.05)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])

In [ ]:
# Plot Model Accuracy and Loss with respect to Train and Validation Datasets
display_training_curves(
    history.history['loss'],
    history.history['val_loss'],
    'loss',
    211,
)
display_training_curves(
    history.history['sparse_categorical_accuracy'],
    history.history['val_sparse_categorical_accuracy'],
    'accuracy',
    212,
)

In [ ]:
# Make predictions and submission to Kaggle
test_ds = generate_test_df(ordered=True) # since we are splitting the dataset and iterating separately on images and ids, order matters.

print('Computing predictions...')
test_images_ds = test_ds.map(lambda image, idnum: image)
probabilities = model.predict(test_images_ds)
predictions = np.argmax(probabilities, axis=-1)
print(predictions)

print('Generating submission.csv file...')
test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(test_count))).numpy().astype('U') # all in one batch
np.savetxt('submission.csv', np.rec.fromarrays([test_ids, predictions]), fmt=['%s', '%d'], delimiter=',', header='id,label', comments='')
